In [1]:
using ColoringNames

runnum = "1"
function mklogdir(name)
    logdir = joinpath(pwd(), "logs","pointest", name * runnum)
    mkpath(logdir)
    logdir
end

mklogdir (generic function with 1 method)

In [2]:
many_names = load_color_nameset()
word_embs, vocab, enc = load_text_embeddings(keep_words=many_names)
cldata = load_munroe_data(dev_as_train=false, dev_as_test=true, encoding_ = enc)

ColoringNames.ColorDatasets{MLLabelUtils.LabelEnc.NativeLabels{String,2833},ColoringNames.ColorDataset{Array{Any,1},Array{Int64,2},Array{Float32,2}}}(MLLabelUtils.LabelEnc.NativeLabels{String,2833}(String["and", "of", "a", "with", "'s", "not", "-", "your", "just", "can"  …  "Feijoa", "melanzane", "minsk", "Pizazz", "feijoa", "red1", "Kilamanjaro", "karaka", "bilbao", "mabel"], Dict("Apple"=>157,"Caper"=>2160,"chocolate"=>424,"Casablanca"=>1155,"Many"=>84,"Luxor"=>1613,"star"=>103,"canyon"=>900,"bombay"=>2563,"bondi"=>2662…)), ColoringNames.ColorDataset{Array{Any,1},Array{Int64,2},Array{Float32,2}}(Any["acid green", "acid green", "acid green", "acid green", "acid green", "acid green", "acid green", "acid green", "acid green", "acid green"  …  "yuck", "yuck", "yuck", "yuck", "yuck", "yuck", "yuck", "yuck", "yuck", "yuck"], [338 338 … 2148 2148; 114 114 … 0 0; 0 0 … 0 0; 0 0 … 0 0], Float32[0.344113 0.855319 0.921569; 0.281905 0.744681 0.921569; … ; 0.376894 0.768559 0.898039; 0.179104 0.

In [3]:
function qualitative_demo(mdl, do_oov=false)
    names = ["black", "brownish green", "brown", "brownish", "greenish", "greenish brown", "green", "red", "orange", "blue"]
    if do_oov
        append!(names, ["ish"])
    end
    plot_query(mdl, names) |> display
end

qualitative_demo(mdl::TermToColorPointEmpirical) = qualitative_demo(mdl, false)

qualitative_demo (generic function with 3 methods)

In [4]:
####################################################

In [5]:
noml = ColoringNames.TermToColorPointEmpirical()
train!(noml, cldata)

qualitative_demo(noml)
@show evaluate(noml, cldata.dev)

black     |brownish green|brown     |brownish  |greenish  |greenish brown|green     |red       |orange    |blue      
evaluate(noml, cldata.dev) = 0.06635567f0


0.06635567f0

In [6]:
sowe = TermToColorPointSOWE(cldata.encoding, word_embs; n_steps=4)
train!(sowe, cldata;
    log_dir=mklogdir("sowe")
)
qualitative_demo(sowe)
@show evaluate(sowe, cldata.dev)

2018-05-30 15:16:47.729570: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2


es_loss = 0.08584482f0
es_loss = 0.06677792f0


es_loss = 0.06683245f0
black     |brownish green|brown     |brownish  |greenish  |greenish brown|green     |red       |orange    |blue      |ish       
evaluate(sowe, cldata.dev) = 0.06683245f0


0.06683245f0

In [7]:
cnn = TermToColorPointCNN(cldata.encoding, word_embs; n_steps=4)

train!(cnn, cldata;
    log_dir=mklogdir("cnn"),
)
qualitative_demo(cnn)
@show evaluate(cnn, cldata.dev)

es_loss = 0.0914553f0
es_loss = 0.06649372f0


es_loss = 0.06670075f0
black     |brownish green|brown     |brownish  |greenish  |greenish brown|green     |red       |orange    |blue      |ish       
evaluate(cnn, cldata.dev) = 0.06670075f0


0.06670075f0

In [8]:
qualitative_demo(cnn)
@show evaluate(cnn, cldata.dev)

black     |brownish green|brown     |brownish  |greenish  |greenish brown|green     |red       |orange    |blue      |ish       
evaluate(cnn, cldata.dev) = 0.06670075f0


0.06670075f0

In [9]:
rnn = TermToColorPointRNN(cldata.encoding, word_embs; n_steps=4)

train!(rnn, cldata;
    log_dir=mklogdir("rnn"),
)
qualitative_demo(rnn)
@show evaluate(rnn, cldata.dev)

es_loss = 0.09244207f0
es_loss = 0.07055541f0
es_loss = 0.070468165f0
es_loss = 0.070466354f0


es_loss = 0.07074563f0
black     |brownish green|brown     |brownish  |greenish  |greenish brown|green     |red       |orange    |blue      |ish       
evaluate(rnn, cldata.dev) = 0.07074563f0


0.07074563f0

In [10]:
1

1

In [11]:
using ColoringNames: order_relevant_dataset, order_relevant_name_pairs

In [14]:
namepairs = order_relevant_name_pairs(cldata.dev);

In [50]:
hsv1s = Array{Float64}((length(namepairs), 3))
hsv2s = similar(hsv1s)
for (ii, (n1, n2)) in enumerate(namepairs)
    hsv1s[ii, :] = query(noml, [n1])
    hsv2s[ii, :] = query(noml, [n2])
end
selectperm(ColoringNames.hsv_squared_error(hsv1s, hsv2s), 1:5)

5-element Array{Int64,1}:
 12
 15
 13
 14
 16

In [51]:
namepairs[[12, 15, 13, 14, 16]]

5-element Array{Array{Any,1},1}:
 Any["blue-purple", "blurple"]        
 Any["brown-grey", "grey-brown"]      
 Any["orange-yellow", "yellow-orange"]
 Any["brown-orange", "orange-brown"]  
 Any["pink-purple", "purple-pink"]    

In [54]:
plot_query(mdl, "pink-purple") |> display
plot_query(mdl, "purple-pink") |> display
plot_query(mdl, "brown-orange") |> display
plot_query(mdl, "orange-brown") |> display
plot_query(mdl, "orange-yellow") |> display
plot_query(mdl, "yellow-orange") |> display
plot_query(mdl, "orange-yellow") |> display
plot_query(mdl, "yellow-orange") |> display

Set(Any["light olive green", "turtle green", "algae", "shamrock", "amber", "ocean", "purple-brown", "soft green", "dusky rose", "khaki green"  …  "earth", "blue-green", "faded blue", "light brown", "dusky purple", "eggplant", "blurple", "dusty pink", "dull yellow", "light orange"])

In [63]:
'-' .∈ collect(names) |> any

false